In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<link rel="stylesheet" href="../../../common/dhds.css">
<div class="Table">
    <div class="Row">
        <div class="Cell grey left"> <img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_portada.jpg" align="center" width="70%"/></div>
        <div class="Cell right">
            <div class="div-logo"><img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/common/logo_DH.png" align="center" width=70% /></div>
            <div class="div-curso">DATA SCIENCE</div>
            <div class="div-modulo">MÓDULO 5</div>
            <div class="div-contenido">Clases desbalanceadas / Feature Selection

</div>
        </div>
    </div>
</div>

### Agenda

---

- Clases desbalanceadas. Qué son. Métodos para resolverlo: Undersampling. Oversampling. Class weighting.
    
- La maldición de la dimensionalidad. Alta dimensionalidad.

- Feature selection. Features con baja varianza. Filter Methods.  Wrapper Methods. 

- Algoritmos genéticos.

<div class="div-dhds-fondo-1"> Clases desbalanceadas
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

### Introducción

---

En los problemas de clasificación a veces encontramos datasets donde alguna de sus clases es  **minoritaria**, porque disponemos pocos casos. 

Esto provoca un **desbalanceo en las clases** (*imbalanced data*), no siempre detectado por los modelos, que tratan de predecir correctamente a las clases mayoritarias.

Algunos ejemplos se encuentran en el diagnóstico de enfermedades donde solemos encontrar miles de registros con *pacientes negativos* y unos pocos *casos positivos*.

En los modelos de detección de fraude tenemos muchas muestras de clientes *honestos* y pocos casos etiquetados como *fraudulentos*, justamente los que necesitamos predecir.

Supongamos que entrenamos un modelo de clasificación con 990 imágenes de gatitos y sólo 10 de perros.

Lo más probable que la red se limite a *responder siempre “tu foto es un gato”*, pues logra **un acierto del 99%** en su fase de entrenamiento.

Lo podemos graficar en la matriz de confusión que generaría el modelo.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_001_matrix.JPG" alt="matrix" width=50% height=40% />

<p style="font-size:70%;">https://www.aprendemachinelearning.com/clasificacion-con-datos-desbalanceados/</p>

### Introducción

---

Veremos tres técnicas para combatir el desbalance de las clases:

- Hacer un resampling: 

    - Aumentando los casos de la clase minoritaria. **Oversampling**.
    
    - Descartando casos de la clase mayoritaria. **Undersampling**.

- Equilibrar la clase minoritaria dandole un peso mayor. **Class weighting**.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_002_resampling.png" alt="resampling" width=55% height=40% />

###  Dataset

---
Vamos a trabajar con datos de incumplimientos de los clientes de tarjetas de crédito.

Las **observaciones** son *clientes* de tarjetas de crédito de Taiwán.

La **clase** representa si la persona está en incumplimiento (1) o no (0).

Para consultar detalles del dataset y sus atributos ver <a href="https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients">aquí</a>.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

%matplotlib inline

In [ ]:
df_original = pd.read_excel('../Data/default_credit_card_clients.xls')
df_original.head(2)

Convertimos las variables categóricas en variables dummies o indicadoras.

In [ ]:
df = pd.get_dummies(df_original,columns=['EDUCATION','MARRIAGE'],drop_first=True);
df.head(5)

Se observa que las clases están **desbalanceadas**:

In [ ]:
print(pd.value_counts(df['Class'], sort = True, normalize=True))

### Modelo

---
Vamos a aplicar en todos los casos un modelo de Regresión Logística, y mediremos su performance.

Preparamos los datos:

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state = 123)

scaler=StandardScaler()  
X_train_sc=scaler.fit_transform(X_train) # Estandarizamos los datos     
X_test_sc=scaler.transform(X_test)

In [ ]:
# definimos una función que crea el modelo que usaremos cada vez
def run_model(X_train, y_train):
    clf_base = LogisticRegression(penalty='none',random_state=123,max_iter=500)
    clf_base.fit(X_train, y_train)
    return clf_base

In [ ]:
# definimos una función para mostrar los resultados
def mostrar_resultados(y_test, pred_y):
    conf_matrix = confusion_matrix(y_test, pred_y); 
    
    plt.figure(figsize=(5, 3))
    sns.heatmap(conf_matrix,  annot=True, fmt="d");
    plt.title("Confusion matrix"); plt.ylabel('True class'); plt.xlabel('Predicted class')
    plt.show()

    print("METRICS")
    print (classification_report(y_test, pred_y))

### Modelo sin modificar el dataset

---
Apliquemos el modelo sobre el dataset original.

Observamos un *aceptable accuracy (0.85)*. Pero los valores de *precision y recall* nos muestran que el modelo es *bueno prediciendo la clase mayoritaria*, pero **muy malo para la clase minoritaria**. 

La matriz de confusión también nos indica un *valor alto de casos mal clasificados para la clase 1*.

In [ ]:
model_original = run_model(X_train_sc, y_train)
y_pred = model_original.predict(X_test_sc)
mostrar_resultados(y_test, y_pred)

### Undersampling

---

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler">imbalanced-learn</a> es un módulo Python que ayuda a balancear los datasets.

Veamos primero como realiza <span style='color: #FF0000;'>undersampling</span>, es decir, como genera un *subset de datos de **train** con clases balanceadas*, **descartando casos de la clase mayoritaria**. 

El método <a href="https://imbalanced-learn.org/stable/under_sampling.html">RandomUnderSampler</a> toma observaciones de la clase mayoritaria al azar, con o sin reposición.

Tiene un hiperparámetro `sampling_strategy` que toma los valores:

* 'minority': hacer resample hasta balancear las clases.
* número entre 0 y 1: ratio entre las clases mayoritaria y minoritaria.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
undersampler=RandomUnderSampler(sampling_strategy='majority',random_state=123); # iguala las clases

X_train_us,y_train_us=undersampler.fit_resample(X_train,y_train);

scaler=StandardScaler()  
X_train_us_sc=scaler.fit_transform(X_train_us) # Estandarizamos los datos     
X_test_us_sc=scaler.transform(X_test)

Observemos que el nuevo train set, al usar `sampling_strategy='majority'`, tiene **la misma cantidad de observaciones por clase**.

Disminuyó la cantidad de observaciones en la clase mayoritaria, hasta obtener una cantidad igual al total original de la clase minoritaria.

In [ ]:
print('NUEVA Composición del training set:\n', y_train_us.value_counts())

In [ ]:
print('ANTERIOR Composición del training set:\n', y_train.value_counts())

### Modelo aplicando Undersampling

---
Apliquemos el modelo sobre este nuevo dataset.

Observamos un descenso de *accuracy* desde 0.85 a 0.69, pues al **reducir fuertemente la cantidad de casos de la clase mayoritaria**, desmejora la predicción para esta clase. La baja en el *recall* de la clase mayoritaria también nos indica este problema.

Por el contrario, la clase minoritaria (1) aumentó el *recall* a 0.66, pues predice mejor a los casos reales. Sin embargo, *precision* bajó de 0.69 a 0.30, mostrando un alto número de falsos positivos.

In [ ]:
model_us = run_model(X_train_us_sc, y_train_us)
y_pred = model_us.predict(X_test_us_sc)
mostrar_resultados(y_test, y_pred)

### Oversampling

---

Ahora probamos lo contrario, <span style='color: #FF0000;'>oversampling</span>, que significa generar un *subset de datos de **train** con clases balanceadas*, **aumentando los casos de la clase minoritaria**. 

El método <a href="https://imbalanced-learn.org/stable/over_sampling.html">RandomOverSampler</a> crea observaciones de la clase minoritaria con reposición.

Con el hiperparámetro `sampling_strategy` realizamos lo mismo que antes con undersampling.

In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversampler=RandomOverSampler(sampling_strategy='minority',random_state=123); # iguala las clases

X_train_os,y_train_os=oversampler.fit_resample(X_train,y_train);

scaler=StandardScaler()  
X_train_os_sc=scaler.fit_transform(X_train_os) # Estandarizamos los datos     
X_test_os_sc=scaler.transform(X_test)

Observemos que el nuevo train set, al usar `sampling_strategy='majority'`, nuevamente tiene **la misma cantidad de observaciones por clase**.

Pero ahora igualó la cantidad de la clase mayoritaria con casos repetidos de la clase minoritaria.

In [ ]:
print('NUEVA Composición del training set:\n', y_train_os.value_counts())

In [ ]:
print('ANTERIOR Composición del training set:\n', y_train.value_counts())

### Modelo aplicando Oversampling

---
Aplicando el modelo sobre este nuevo dataset, vemos que la performance con oversampling es prácticamente igual a la obtenida con el dataset generado con *undersampling*.

No nos olvidemos que *podemos jugar con valores entre 0 y 1* sobre el parámetro `sampling_strategy` para mejorar la performance.

In [ ]:
model_os = run_model(X_train_os_sc, y_train_os)
y_pred = model_os.predict(X_test_os_sc)
mostrar_resultados(y_test, y_pred)

### Oversampling - SMOTE

---
El algoritmo <a href="https://imbalanced-learn.org/stable/references/over_sampling.html#smote-algorithms">SMOTE</a> (Synthetic Minority Oversample Technique) realiza oversampling, generando **muestras simuladas de la clase minoritaria**.

La idea es considerar de a pares casos de la clase minoritaria, y generar aleatoriamente un punto que se encuentre en el segmento que los une.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_003_smote.png" alt="smote" width=55% height=40% />

Pero *SMOTE* solo funciona con **variables continuas**.

Por suerte, existe el método <a href="https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTENC.html">SMOTENC</a> (NC por Nominal and Continuous) que trabaja con **features continuas y categóricas**.

Tenemos que informar las variables categóricas como una lista con la posición de cada feature en el dataset, en el parámetro `categorical_features`.

In [ ]:
from imblearn.over_sampling import SMOTENC
smote=SMOTENC(categorical_features=[1, 2, 4, 5, 6, 7, 8, 9],sampling_strategy='minority',random_state=123);

X_train_sm,y_train_sm = smote.fit_resample(X_train,y_train);

scaler=StandardScaler()  
X_train_sm_sc=scaler.fit_transform(X_train_sm) # Estandarizamos los datos     
X_test_sm_sc=scaler.transform(X_test)

Observemos que el nuevo train set tiene la misma cantidad de observaciones por clase.

Pero ahora igualó la cantidad de la clase mayoritaria con **casos inventados de la clase minoritaria**.

In [ ]:
print('NUEVA Composición del training set:\n', y_train_sm.value_counts())

### Modelo aplicando SMOTENC

---
En este modelo, la performance sigue prácticamente igual a las obtenidas anteriormente.

In [ ]:
model_sm = run_model(X_train_sm_sc, y_train_sm)
y_pred = model_sm.predict(X_test_sm_sc)
mostrar_resultados(y_test, y_pred)

### Class weighting

---

Otra técnica que se encuentra en muchos algoritmos de machine learning es **penalizar o darle un peso a cada label de la clase**.

Se puede usar para imponer un costo al modelo cuando falla al clasificar la clase minoritaria, balanceando las clases.

La regresión logística admite el hiperparámetro `class_weight`, un diccionario donde se definen los pesos por label.

Con `class_weight = 'balanced'` los pesos asignados son la inversa de la frecuencia de cada label.

Es otra forma de balancear las clases, sin hacer resample.

In [ ]:
model = LogisticRegression(class_weight = 'balanced',penalty='none',random_state=123,max_iter=500)

scaler=StandardScaler()

X_train_sc=scaler.fit_transform(X_train);
X_test_sc=scaler.transform(X_test);

model.fit(X_train_sc,y_train)
y_pred = model.predict(X_test_sc)
mostrar_resultados(y_test, y_pred)

### Resumen

---
Los modelos de Machine Learning generalmente están sesgados hacia la clase mayoritaria. Y en general requerimos predecir correctamente la clase minoritaria.

Resolver esta problemática es relevante, pues se presentan en muchas situaciones de la vida real: análisis financieros (fraudes, Churn, Default), análisis biomédicos (outliers), seguridad (intrusión), etc.

Todas las técnicas que vimos ayudan a *mejorar el recall*. Debemos probarlas, modificar sus parámetros, para llegar a la mejor performance para el dataset que estamos trabajando.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_005_tabla.JPG" alt="tabla" width=100% height=80% />

<div class="div-dhds-fondo-1"> La maldición de la dimensionalidad
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

### Introducción

---
A priori, podemos pensar que una mayor cantidad de features mejora los modelos, pero no necesariamente es así.

Definimos a un dataset como de **alta dimensionalidad**, si tiene un número de features del orden de *cientos o más*. 

En general, al agregar features *se mejora hasta cierto punto o umbral*, a partir del cual el clasificador comienza a empeorar su performance.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_006_grafico_alta_dim.png" alt="grafico_alta_dim" width=40% height=35% />

La frase, atribuida a Richard Bellman, **La maldición de la dimensionalidad** refiere a los problemas de entrenamiento de modelos debidos a una alta dimensionalidad.


### Ejemplo

---

Veamos con un simple ejemplo el efecto de agregar dimensiones a un dataset que **no le incrementamos la cantidad de observaciones**.

Supongamos que tenemos diez imágenes de gatos y perros usando el modelo de color RGB, que los representa con los colores *Rojo, Verde y Azul*.

Y tenemos que construir un clasificador en base a estas imágenes.

Si usamos solo el *feature “rojo”* nuestro clasificador no funcionará bien.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_007_ejemplo.png" alt="grafico_alta_dim" width=30% height=25% />

Agreguemos la *feature “verde”* y las observaciones se empiezan a separar.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_008_ejemplo.png" alt="grafico_alta_dim" width=30% height=25% />

Consideremos la *tercera feature “azul”*. Logramos mayor dispersión.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_009_ejemplo.png" alt="grafico_alta_dim" width=30% height=25% />

Ahora podemos encontrar un plano que separa perfectamente gatos y perros y que es la combinación lineal de “rojo”, “verde” y “azul”.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_010_ejemplo.png" alt="grafico_alta_dim" width=30% height=25% />

Hasta acá parece que agregar features mejora la capacidad de clasificación del modelo.

### Ejemplo

---
Al agregar features, la dimensionalidad del espacio de predictores se incrementa y **los datos se hacen cada vez más dispersos (sparse)**.

Es cada vez *más fácil encontrar un hiperplano que separe a las clases*, porque la probabilidad de encontrar un dato mal clasificado (del lado equivocado del hiperplano) se hace infinitamente pequeña.

**Y corremos el riesgo de caer en el overfitting**.

En este ejemplo, un clasificador lineal en dos dimensiones, **es más simple y generaliza mejor** que el clasificador lineal en tres dimensiones.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_011_ejemplo.png" alt="grafico_alta_dim" width=30% height=25% />

### Proporción de outliers

---
Otro problema es que, al crecer la dimensionalidad del problema, cada vez más casos se ubican en los extremos del espacio.

El número de outliers aumenta porque, *al agregar dimensiones, aumenta la probabilidad* de que <b>al menos en una dimensión </b> la variable tome un valor extremo.

Al separarse los puntos en el espacio de muchas dimensiones, **se puede perder la estructura que buscamos encontrar en un espacio demasiado grande**.

### Ejemplo

---
Vamos a comprobarlo:

Dada una variable distribuida uniformemente en un hipercubo de $d$ dimensiones, *veamos como crecen los outliers*.

Los definimos como aquellos puntos que toman valores extremos en alguna de las $d$ dimensiones. Es decir, si **alguna de sus coordenadas** está en el percentil 1 o 99 del eje.

Con dos dimensiones, creamos un marco cuadrado. Encontramos unos pocos.

In [ ]:
N=500;

X=np.random.uniform(size=(N,2)); # generamos 500 puntos en un espacio de dos dimensiones, distribuidos uniformemente

plt.figure(figsize=(5, 3)); plt.plot(X[:,0],X[:,1],'o',ms=2);x = [0.0,1.0]
p = 0.01 # Si alguna de las coordenadas está en el percentil 1 o 99 del eje, lo consideramos un outlier

plt.fill_between(x,0,p,alpha=0.2,color='k'); plt.fill_between(x,1-p,1,alpha=0.2,color='k')
plt.fill_betweenx(x,0,p,alpha=0.2,color='k'); plt.fill_betweenx(x,1-p,1,alpha=0.2,color='k',label='Region de outliers');
plt.legend(loc=(0,1)); plt.show()

n_outliers=np.sum(np.sum((X <p) |( X >(1-p)),axis=1));
print('Número de outliers:',n_outliers, ' de ',N)

### Ejemplo

---
Aumentando la dimensionalidad del dataset, *para un número fijo de casos*, vemos **como crece la proporción de outliers**.

In [ ]:
N=1000; p=0.01; Ds=np.arange(1,200);
p_outliers=[];
for d in Ds:
    X=np.random.uniform(size=(N,d));
    p_outliers.append(np.mean(np.any((X <p) |( X >(1-p)),axis=1)))

plt.figure(figsize=(6, 4)); plt.plot(Ds, p_outliers); 
plt.ylim([0,1]); plt.ylabel('Proporción de outliers'); plt.xlabel('Dimensiones');

<div class="div-dhds-fondo-1"> Feature Selection
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

### Introducción

---
Dado el punto anterior, como podemos resolver los problemas cuando tenemos alta dimensionalidad?

Evidentemente, **debemos reducir el número de features**.

¿Pero cómo seleccionamos el subset de features que optimiza la performance de nuestro modelo?

Un enfoque de *fuerza bruta* haría una búsqueda exhaustiva de todas las combinaciones posibles de features para encontrar el mejor conjunto. *Impracticable en términos computacionales*.

**Feature Selection** busca identificar y seleccionar las variables más relevantes para el entrenamiento de los modelos. 

También se puede ver como el proceso enfocado en *remover las variables que no traen información o son redundantes*.

Veamos algunos <a href="https://scikit-learn.org/stable/modules/feature_selection.html#">métodos implementados en Scikit-Learn</a>.

### Dataset

---
Vamos a trabajar con un archivo con predicciones de supervivencia de pacientes que tuvieron ataques cardiacos previamente.

- Tenemos variables categóricas que indican factores de riesgo (1): *anemia, diabetes, high_blood_pressure, smoking*.

- *sex*, indica 0 - mujer, 1 - hombre. *age*, la edad del paciente.

- *creatinine_phosphokinase, platelets, serum_creatinine, serum_sodium* son medidas de componentes de la sangre.

- *ejection_fraction* mide como trabaja el corazón. *time*, los días de seguimiento.

- *DEATH_EVENT* es la variable target donde 0 - indica supervivencia, 1 - falleció.

Para más detalles ver <a href="https://www.kaggle.com/andrewmvd/heart-failure-clinical-data">aquí</a>.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

%matplotlib inline

In [ ]:
df_heart = pd.read_csv('../Data/heart_failure_clinical_records_dataset.csv')
print('Total de filas: ',df_heart.shape[0],'Total de columnas: ',df_heart.shape[1])
df_heart.head(8)

### Filter Methods

---
Buscan rankear las features en función de su **"importancia”**.

Habitualmente, se define un *umbral* para los scores, por debajo del cual las variables son consideradas poco relevantes y se filtran.

Los métodos varían de acuerdo a *como miden la relevancia* de las features. 

Sólo **después** de encontrar las mejores features, se puede generar los modelos.

Veremos algunas implementaciones en <a href="https://scikit-learn.org/stable/modules/feature_selection.html#">Scikit-Learn</a>.

### Features con baja varianza

---
La clase <a href="https://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance">VarianceThreshold</a> remueve las features que **no aporten información a nuestro dataset**, calculando su varianza.

Es decir, se filtran todos las features que tengan una varianza (variabilidad) menor a un `threshold` (umbral) definido.

---
Vamos a aplicarlo a nuestro dataset.

Calculamos primero la varianza de cada feature, para validarlo después con el método. Las primeras siete features superan un valor de 0.5, es cual será el umbral que usaremos con `VarianceThreshold`.

In [ ]:
pd.options.display.float_format = '{:.6f}'.format
df_heart.apply(np.var).sort_values(ascending=False)[:8]

Ajustamos el dataset a la clase, con el hiperparámetro `threshold` = 0.5.

`fit_transform` devuelve un array donde las columnas son solo las features seleccionadas.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=0.5) # Instanciamos la clase con un threshold=0.5
df_heart_vt = vt.fit_transform(df_heart)
df_heart_vt[:2]

El método `get_support()` devuelve una máscara con las features seleccionadas.

In [ ]:
vt.get_support()

Si lo aplicamos al dataset podemos conocer cuales son.

In [ ]:
df_heart.columns[vt.get_support()]

Finalmente, podemos reconstruir un dataframe solo con las features seleccionadas.

In [ ]:
df_heart_reduced_vt = pd.DataFrame(df_heart_vt, columns = df_heart.columns[vt.get_support()])
df_heart_reduced_vt.sample(4)

### Selección de features univariados 

---
Los métodos de *Univariate feature selection* realizan un ranking de las features a partir de tests estadísticos univariados.

Analizan sus propiedades estadísticas y *cuan fuerte es su relación con la variable target*. 

Son *univariados* pues no contemplan *interacción entre features*. Se evalúa una variable a la vez con la clase.

Los tests estadísticos más usados son:
- Pearson’s Correlation Coefficient. Llamado como la función `f_regression()`
- ANOVA: `f_classif()`
- Chi-Squared: `chi2()`
- Mutual Information: `mutual_info_classif()` and `mutual_info_regression()`

Veamos dos métodos que implementa <a href="https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection">Scikit-learn</a>:

- SelectKBest.
- SelectPercentile.

### SelectKBest 

---
La clase <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest">SelectKBest</a> usa una suite de test estadísticos para seleccionar las features más significativas.

Necesita dos hiperparámetros:

* `score_func`: una función que devuelve algún score entre $X$ (las features) e $Y$ (la variable target).

* `k`: la cantidad de "mejores" features que deben ser seleccionadas.

Como vimos, las funciones implementan los tests estadísticos. Para clasificación son f_classif, mutual_info_classif, chi2. Para regresión, f_regression, mutual_info_regression.

### Ejemplo 

---
Vamos a usar el *test estadístico Chi-cuadrado* para seleccionar las 10 mejores features del dataset.
* `score_func` = chi2.
* `k` = 10.

Previamente generamos la matriz de features y la variable target.

In [ ]:
X = df_heart.drop('DEATH_EVENT',axis=1)
y = df_heart['DEATH_EVENT']

El método `fit_transform` aplica el test estadístico y selecciona las 10 features más relevantes, devolviéndolas como un array.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

bestfeatures_k = SelectKBest(score_func=chi2, k=10)
fit_k = bestfeatures_k.fit_transform(X,y)

Reconstruimos un dataframe con las 10 features seleccionadas.

In [ ]:
df_heart_reduced_k = pd.DataFrame(fit_k, columns = X.columns[bestfeatures_k.get_support()])
df_heart_reduced_k.sample(4)

Se puede ver el score de cada feature.

In [ ]:
dfscores = pd.DataFrame(bestfeatures_k.scores_)
dfcolumns = pd.DataFrame(X.columns)

scores = pd.concat([dfcolumns,dfscores],axis=1)
scores.columns = ['Feature','Score']

print(scores.nlargest(10,'Score'))

### SelectPercentile

---
La clase <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile">SelectPercentile</a> es similar a `SelectKBest`, pero selecciona los features en base a los percentiles de los scores. Por ejemplo, los que se encuentran en el mejor 20%.

Necesita dos hiperparámetros, el primero similar a `SelectKBest`:

* `score_func`: una función que devuelve algún score entre $X$ (las features) e $Y$ (la variable target).

* `percentile`: porcentaje de features a considerar.

Repetimos el ejemplo, para un 20%. 
* `score_func` = chi2.
* `percentile` = 20.

El método `fit_transform` aplica el test estadístico y selecciona solo el 20% más relevante de features. En nuestro ejemplo son tres features de las 12 del dataset.

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2

bestfeatures_p = SelectPercentile(chi2, percentile=20)
fit_p = bestfeatures_p.fit_transform(X,y)

El método `get_support()` devuelve una máscara con las features seleccionadas.

In [ ]:
bestfeatures_p.get_support()

Reconstruimos un dataframe solo con las features seleccionadas.

In [ ]:
df_heart_reduced_p = pd.DataFrame(fit_p, columns = X.columns[bestfeatures_p.get_support()])
df_heart_reduced_p.sample(4)

<div class="div-dhds-fondo-1"> Wrapper methods
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

### Wrapper methods

---
Los métodos seleccionan subconjuntos de features *según la performance* que obtienen al ajustarlos sobre un modelo.

Dicho de otra forma, aplica un algoritmo sobre un subconjunto de variables, y de acuerdo a la performance obtenida, *remueve o agrega features*. Luego **vuelve a aplicar recursivamente el algoritmo** hasta llegar a un criterio de parada.

Si bien es un proceso más costoso en términos computacionales, **son más exactos** que los *Filter methods*, ya que se evalúan sobre un modelo real. 

Wrapper methods consideran a la selección del conjunto de features como un problema de búsqueda, donde se aplican distintas estrategias.

Diferencias entre ambos tipos de métodos

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_013_diferencias.JPG" alt="diferencias" width=70% height=60% />

### Eliminación Recursiva de Features (RFE)

---
*Recursive feature elimination* entrena un **estimador** sobre el total de features y calcula la importancia de cada feature. La feature menos importante *se elimina del set* y se vuelve a entrenar con los restantes. 

El proceso se repite de forma recursiva hasta que se llega al **número de features definido previamente**.

El método `RFE()` tiene los argumentos:

* `estimator`: el estimador usado para entrenar y evaluar. Puede ser cualquier algoritmo supervisado que devuelva la importancia de cada feature.
* `n_features_to_select`: la cantidad de features final.
* `steps`: features que se eliminan por iteración.

Apliquemos el método sobre el dataset de vinos, que tiene como features sus propiedades químicas, y como variable target una clasificación en tres tipos de vinos (0,1,2).

In [ ]:
from sklearn.datasets import load_wine
X,y = load_wine(as_frame=True, return_X_y=True)
print('Total de filas: ',X.shape[0],'Total de columnas: ',X.shape[1])
X.head(2)

Estandarizamos las variables. Pero no hace falta dividir en train y test, ya que usamos el algoritmo para validar las features.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()    
X_sc = scaler.fit_transform(X)

Usamos como clasificador un árbol de decisión.

Le informamos, con el hiperparámetro `n_features_to_select` que nos quedamos con 6 features. Y con `step` le decimos que elimine de a uno.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

estimator = DecisionTreeClassifier()
rfe = RFE(estimator, n_features_to_select=6, step = 1)
rfe.fit(X_sc, y)

scores = pd.DataFrame()
scores["Attribute Name"] = X.columns; scores["Ranking"] = rfe.ranking_; scores["Support"] = rfe.support_

print(scores.sort_values('Ranking'))

### Eliminación Recursiva de Features con Cross Validation (RFECV)

---
*Recursive feature elimination with cross-validation* es similar a *RFE*, pero **determina la cantidad de features a seleccionar usando CrossValidation**.

Los argumentos son similares, pero *ya no informamos cuantas variables finales queremos*. Y por otra parte, le tenemos que decir como hacemos el CV. 

El método `RFECV()` tiene los argumentos:

* `estimator`: el estimador usado para entrenar y evaluar. Puede ser cualquier algoritmo supervisado que devuelva la importancia de cada feature.
* `cv`: determina el método de validación cruzada.
* `steps`: features que se eliminan por iteración.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

estimator = DecisionTreeClassifier()
kf = StratifiedKFold(n_splits = 5, shuffle = True)

rfecv = RFECV(estimator, cv=kf, step = 1)
rfecv.fit(X_sc, y)

scores = pd.DataFrame()
scores["Attribute Name"] = X.columns; scores["Ranking"] = rfecv.ranking_; scores["Support"] = rfecv.support_

print(scores.sort_values('Ranking'))


<div class="div-dhds-fondo-1"> Algoritmos genéticos
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

### Introducción

---
Fueron creados por John Holland en los años 1970. Son llamados así porque se inspiran en la evolución biológica y su base genético-molecular.

Básicamente es un método de búsqueda y optimización.

Donde el espacio de búsqueda de un problema determinado está formado por todas las posibles soluciones a dicho problema.

Tomemos como ejemplo:

El problema del viajante (TSP por sus siglas en inglés Travelling Salesman Problem) responde a la siguiente pregunta: 



**Dada una lista de ciudades y las distancias entre cada par de ellas, ¿cuál es la ruta más corta posible que visita cada ciudad exactamente una vez y al finalizar regresa a la ciudad origen?**

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_014_ag_ejemplo.JPG" alt="ag_ejemplo" width=50% height=40% />

donde tenemos soluciones malas y buenas.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_015_ag_ejemplo.JPG" alt="ag_ejemplo" width=70% height=60% />

El espacio de búsqueda *son todos los posibles caminos sobre el grafo*, que pasen, para simplificar, solo una vez por cada ciudad. La cantidad de caminos es el factorial de 6 (6! = 720).

Podemos decir que cada camino es **una solución dentro del espacio total de soluciones**.

Y cada camino se puede representar como un vector de 6 elementos, donde cada uno de ellos representa una ciudad.

$(x_1,x_2,x_3,x_4,x_5,x_6)$ donde $x_i$ es una ciudad entre 1 y 6, y $x_i \ne x_j$.

Entonces $(1,2,3,4,5,6)$ es el camino óptimo.

### Algoritmo

---
Los algoritmos genéticos trabajan sobre una población de individuos. *Cada individuo es una solución dentro del espacio de soluciones*.

Evolucionan la población (se mueven de un conjunto de individuos a otros) aplicando acciones semejantes a las que actúan en la evolución biológica. *"El proceso de selección natural”*.

Cada iteración o **generación** encuentra nuevos individuos "más aptos".

El algoritmo termina cuando encuentra la mejor solución, *los mejores individuos*, **no siempre los óptimos**, o bien luego de un número de generaciones.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_016_ag_espacio_sol.JPG" alt="ag_espacio_sol" width=60% height=45% />

Cada individuo de la población está representado por un **cromosoma**, *un vector*, compuesto por elementos a los que llamamos **genes**.

Las diferentes formas (valores) que puede tomar un gen se denomina **alelo**. 

La posición física que ocupa cada gen dentro del cromosoma se denomina **locus**.

Todos los cromosomas conforman la **población**, o dicho de otra forma, *el espacio de soluciones*.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_017_ag_individuo.png" alt="ag_individuo" width=60% height=45% />

**Como representamos nuestro problema de seleccionar las mejores features?**

Podemos asumir que *cada gen indica si está presente la feature i*, con un *alelo* de 0 y 1.

De modo que el *cromosoma es un vector de 13 genes*, cada uno indicando si contiene o no una de las features de la matriz de features del dataset de vinos.

La población es el conjunto de cromosomas, es decir un conjunto de $2^{13}$ elementos.

Viendo el enorme volumen de soluciones, los algoritmos genéticos nos brindan la posibilidad de **una búsqueda heurística** de la mejor solución.

### Fases

---
Veamos la **"evolución"** de los individuos, hasta llegar a los  *más aptos*.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_018_ag_circuito.png" alt="ag_circuito" width=90% height=65% />

- **Población inicial**: es un conjunto de individuos de tamaño **size** determinado, seleccionados al azar.

- **Fitness**: es una *medida de performance* del modelo cuando lo ajustamos a los individuos.

  Puede ser accuracy para modelos de clasificación,  𝑟2  para regresión, etc.

  En general, la función de fitness determina *qué tan apto* es un individuo.
  
  Nos quedamos con los **n_best cromosomas**. 
  
- **Criterio de terminación**: genera una medida del fitness de los cromosomas, un **score**. 
  Es una *medida de error*; si es menor a cierto threshold (umbral) finaliza el algoritmo.
  
  También puede ser *un número final de iteraciones alcanzadas*.

- **Selección**: a los *n_best cromosomas*, le sumamos *n_rand* cromosomas al azar.
  
  Luego algunos se seleccionan para generar descendencia, y otros siguen a la etapa de mutación.

- **Pairing, Mating**: selecciona parejas de cromosomas para "aparearlas", y generar una nueva descendencia de *n_children* cromosomas.

  Se realiza mediante **crossover**, combinando genes de ambos individuos.
  
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_019_ag_crossover.png" alt="ag_crossover" width=60% height=45% />

- **Mutation**: un porcentaje *cambia el valor en algunos de sus genes*.
  
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_020_ag_mutation.png" alt="ag_mutation" width=60% height=45% />

- **Nueva generación**: los nuevos cromosomas generados se evalúan y se seleccionan nuevamente los n_best cromosomas. Se llama a este proceso *elitismo*.

### Implementación

---
Python no dispone de una librería para los algoritmos genéticos, pero vamos a usar una implementación simple del algoritmo. El código fuente se encuentra en 
[acá](https://github.com/dawidkopczyk/genetic/blob/master/genetic.py).

In [ ]:
%run "../Notebooks/Genetic.py"

Analicemos ahora el problema de clasificación de vinos usando algoritmos genéticos.

Medimos la performance del modelo de clasificación con todas las features, antes de comenzar con el algoritmo.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
est = DecisionTreeClassifier()
cv_param = StratifiedKFold(n_splits = 5, shuffle = True)
score = cross_val_score(est, X_sc, y, cv=cv_param, scoring='balanced_accuracy')
print("Performance con todas las features: {:.2f}".format(np.mean(score)))

La implementación tiene los parámetros:

* *estimator*: el estimador usado para calcular el fitness.
* *n_gen*: el número máximo de generaciones (número de iteraciones del algoritmo).
* *size*: tamaño de la población a considerar.
* *n_best*: en cada iteración se seleccionan los n_best cromosomas.
* *n_rand*: además de los n_best, se seleccionan n_rand cromosomas al azar.
* *n_children*: número de nuevos cromosomas que se generan en cada apareamiento.
* *mutation_rate*: probabilidad de cambiar cada gen espontáneamente.
* *cv*: default 5, argumento para la función cross_val_score de sklearn.
* *scoring*: medida de fitness de los cromosomas. Por ejemplo "neg_mean_squared_error" para modelos de regresión, o "accuracy" para clasificación. Dentro de la clase se multiplica la medida por -1 convirtiéndola en una medida de error.

In [ ]:
sel = GeneticSelector(estimator = DecisionTreeClassifier(), 
                      n_gen = 15, size = 200, n_best = 40, 
                      n_rand = 40, n_children = 5, mutation_rate = 0.05,
                     scoring = 'balanced_accuracy')
sel.fit(X_sc, y)

### Implementación

---
El gráfico nos muestra como fue bajando el error a medida que itera el algoritmo, es decir, que crea nuevas generaciones.

*Average* es el promedio de los scores de los cromosomas de la generación. 

*Best* es el mejor score de cada generación.

In [ ]:
sel.plot_scores()
score = cross_val_score(est, X_sc[:,sel.support_], y, cv = cv_param, scoring='balanced_accuracy')
print("Performance con las features seleccionadas: {:.2f}".format(np.mean(score)))

Veamos finalmente cuales son las features seleccionadas.

In [ ]:
scores = pd.DataFrame()
scores["Attribute Name"] = X.columns; scores["Support"] = sel.support_
print(scores.sort_values('Support', ascending=False))

### Ventajas y desventajas

---
*Ventajas* 

- Más rápido y eficiente que los métodos tradicionales.
- Permite procesamiento paralelo.
- Genera una lista de "buenas" soluciones.
- Util para grandes espacios de soluciones.

*Desventajas*
- No es apropiado para problemas simples.
- No garantiza llegar a la solución óptima.
- La evaluación frecuente del fitness lo hace computacionalmente caro.


<div class="div-dhds-fondo-1"> Conclusiones
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

## Conclusiones

---

* Los datasets con *clases desbalanceadas* se presentan en muchas situaciones de la vida real. Tenemos que transformarlos para que los algoritmos de ML funcionen correctamente.

* *La maldición de la dimensionalidad* se refiere a los problemas que tienen los modelos para entrenar cuando tenemos alta dimensionalidad.

* La reducción de features en los datasets nos lleva a los métodos de *features selection*. 

* Tenemos los *filter methods* que buscan rankear las features en función de un valor de importancia. Habitualmente, se define un umbral para los scores, por debajo del cual las variables son consideradas poco relevantes y se filtran.

* Los *wrapper methods* seleccionan subconjuntos de features según la performance que obtienen al ajustarlos sobre un modelo.

* Podemos adaptar los *algoritmos genéticos* para seleccionar las mejores features.

<div class="div-dhds-fondo-1"> Hands-on
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

### Ejercicio

----

A partir del dataset de vinos, apliquemos los *filter methods*:

- SelectKBest, usando el *test estadístico Chi-cuadrado* para seleccionar las 6 mejores features del dataset.

- SelectPercentile, para seleccionar las features que se encuentran en el mejor 25%.

In [ ]:
from sklearn.datasets import load_wine
X,y = load_wine(as_frame=True, return_X_y=True)
print('Total de filas: ',X.shape[0],'Total de columnas: ',X.shape[1])
X.head(2)

### Solución

---

### Ejercicio

----

A partir del dataset de vinos, apliquemos los *filter methods*:

- SelectKBest, usando el *test estadístico Chi-cuadrado* para seleccionar las 6 mejores features del dataset.

- SelectPercentile, para seleccionar las features que se encuentran en el mejor 25%.

In [ ]:
from sklearn.datasets import load_wine
X,y = load_wine(as_frame=True, return_X_y=True)
print('Total de filas: ',X.shape[0],'Total de columnas: ',X.shape[1])
X.head(2)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### SelectKBest

----
Vamos a usar el *test estadístico Chi-cuadrado* para seleccionar las 6 mejores features del dataset.

* `score_func` = chi2.
* `k` = 6.

El método `fit_transform` aplica el test estadístico y selecciona las 6 features más relevantes. Las deja como un array.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

bestfeatures_k = SelectKBest(score_func=chi2, k=6)
fit_k = bestfeatures_k.fit_transform(X,y)

Reconstruimos un dataframe con las 6 features seleccionadas.

In [ ]:
X_reduced_k = pd.DataFrame(fit_k, columns = X.columns[bestfeatures_k.get_support()])
X_reduced_k.sample(4)

### SelectPercentile

----
Usamos los siguientes parámetros:

* `score_func` = chi2.
* `percentile` = 25.

El método `fit_transform` aplica el test estadístico y selecciona solo el 25% más relevante de features. 

En nuestro ejemplo son tres features de las 12 del dataset.

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2

bestfeatures_p = SelectPercentile(chi2, percentile=20)
fit_p = bestfeatures_p.fit_transform(X,y)

Reconstruimos un dataframe solo con las features seleccionadas.

In [ ]:
X_reduced_p = pd.DataFrame(fit_p, columns = X.columns[bestfeatures_p.get_support()])
X_reduced_p.sample(4)

<div class="div-dhds-fondo-1"> Referencias y Material Adicional
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_40_Feature_Selection_Desbalance_Clases/Presentacion/img/M5_CLASE_40_separador.png" align="center" />

</div>

### Referencias y Material Adicional

---

<a href="https://imbalanced-learn.org/stable/index.html" target="_blank">Imbalanced learn documentation</a>

<a href="https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/" target="_blank">Tactics to combat imbalanced classes</a>

<a href="https://www.aprendemachinelearning.com/clasificacion-con-datos-desbalanceados/" target="_blank">Clasificación con datos desbalanceados</a>

<a href="https://towardsdatascience.com/5-smote-techniques-for-oversampling-your-imbalance-data-b8155bdbe2b5" target="_blank">5  SMOTE techniques for oversampling your imbalance data</a>

<a href="https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e" target="_blank">Feature selection techniques in machine learning with Python</a>

<a href="https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2" target="_blank">The 5 Feature Selection Algorithms every Data Scientist should know</a>

<a href="https://medium.com/@rinu.gour123/python-genetic-algorithms-with-artificial-intelligence-b8d0c7db60ac" target="_blank">Python Genetic Algorithms With Artificial Intelligence</a>